In [1]:
#import sys
#sys.path.append(r'C:\Users\OMISTAJA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages')

import whitebox
import numpy as np
import geopandas as gpd
import os

#### Download Whitebox Tools
This tool needs absolute paths to files to work. 
Code: https://www.whiteboxgeo.com/manual/wbt_book/available_tools/hydrological_analysis.html#ImpoundmentSizeIndex 

Tried first with the resolution of 2m x 2m, worked well. Resolution 0.5 x 0.5 takes too much memory, this was made in the end with 1m resolution (last code in here). 

### 1m resolution
This Impoudment size index was difficult to make for the small resolution data. Every time teh memory was a limiting feature. In the end I decided to make a code driving the area in pieces. 

In [4]:
#import sys
#sys.path.append(r'C:\Users\OMISTAJA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages')

# Initialize WhiteboxTools
wbt = whitebox.WhiteboxTools()

# Set the working directory
wbt.work_dir = "E:/Gradu/ImpoudmentSizeIndex/"
# Input file paths
dem = 'E:/Gradu/ImpoudmentSizeIndex/ISI_DEM_0_5_nn.tif'
reservoirs_shapefile = 'E:/Gradu/ImpoudmentSizeIndex/CA.shp'

In [ ]:
# Optional callback function for monitoring progress
def default_callback(progress):
    print(f"Progress: {progress}%")

#Output file paths, using external memory to save space
out_mean="E:/Gradu/ImpoudmentSizeIndex/ISI/mean_index-test.tif" 
out_max="E:/Gradu/ImpoudmentSizeIndex/ISI/max_index-test.tif"
out_volume="E:/Gradu/ImpoudmentSizeIndex/ISI/volume-test.tif" 
out_area="E:/Gradu/ImpoudmentSizeIndex/ISI/area-test.tif"
out_dam_height="E:/Gradu/ImpoudmentSizeIndex/ISI/dam_height-test.tif"

# Run the function
wbt.impoundment_size_index(
    dem=dem, 
    damlength=9, 
    out_mean=out_mean, 
    out_max=out_max, 
    out_volume=out_volume, 
    out_area=out_area, 
    out_dam_height=out_dam_height, 
    callback=default_callback
)

### Using the 1m (last code here) or 2m resolution to increase resolution to 0.5
gdalwarp -tr 0.5 0.5 -r bilinear dam_height_2m.tif dam_height_0_5m.tif
#use this line in correct directory, or dot his:

In [33]:
import os
from osgeo import gdal

In [34]:
from osgeo import gdal
print(gdal.VersionInfo())

3080400


#### Check functions osgeo - skip, problem solving
Checking out error about not having gdal...

In [36]:
from osgeo import gdal

def test_gdal_functions():
    # Print available GDAL drivers
    drivers = gdal.GetDriverCount()
    for i in range(drivers):
        driver = gdal.GetDriver(i)
        print(f"Driver: {driver.ShortName}, {driver.LongName}")

test_gdal_functions()

Driver: VRT, Virtual Raster
Driver: DERIVED, Derived datasets using VRT pixel functions
Driver: GTiff, GeoTIFF
Driver: COG, Cloud optimized GeoTIFF generator
Driver: NITF, National Imagery Transmission Format
Driver: RPFTOC, Raster Product Format TOC format
Driver: ECRGTOC, ECRG TOC format
Driver: HFA, Erdas Imagine Images (.img)
Driver: SAR_CEOS, CEOS SAR Image
Driver: CEOS, CEOS Image
Driver: JAXAPALSAR, JAXA PALSAR Product Reader (Level 1.1/1.5)
Driver: GFF, Ground-based SAR Applications Testbed File Format (.gff)
Driver: ELAS, ELAS
Driver: ESRIC, Esri Compact Cache
Driver: AIG, Arc/Info Binary Grid
Driver: AAIGrid, Arc/Info ASCII Grid
Driver: GRASSASCIIGrid, GRASS ASCII Grid
Driver: ISG, International Service for the Geoid
Driver: SDTS, SDTS Raster
Driver: DTED, DTED Elevation Raster
Driver: PNG, Portable Network Graphics
Driver: JPEG, JPEG JFIF
Driver: MEM, In Memory Raster
Driver: JDEM, Japanese DEM (.mem)
Driver: GIF, Graphics Interchange Format (.gif)
Driver: BIGGIF, Graphics I

#### Using 2m resolution to increase to 0.5m resolution
My tests to make ImpoudmentSizeIndex from 0.5m resolution data failed. This is increasing resolution from the 2m resolution.,

In [38]:
from osgeo import gdal

def resample_tiff(input_tiff, output_tiff, x_res, y_res):
    dataset = gdal.Open(input_tiff)
    if dataset is None:
        raise FileNotFoundError(f"Input file {input_tiff} not found or unable to open.")
    
    # Perform the warp (resampling to specified resolution)
    gdal.Warp(
        output_tiff,
        dataset,
        xRes=x_res,
        yRes=y_res,
        resampleAlg='bilinear'
    )

    print(f"Resampled {input_tiff} to {output_tiff} with resolution {x_res}x{y_res}.")

# Example usage
input_tiff = 'H:/PurOja/01_Data/01_Raw/Halli_RawData_12_zones/ImpoudmentSizeIndex/dam_height.tif'
output_tiff = 'H:/PurOja/01_Data/01_Raw/Halli_RawData_12_zones/ImpoudmentSizeIndex/dam_height_0_5m.tif'
resample_tiff(input_tiff, output_tiff, 0.5, 0.5)

Resampled H:/PurOja/01_Data/01_Raw/Halli_RawData_12_zones/ImpoudmentSizeIndex/dam_height.tif to H:/PurOja/01_Data/01_Raw/Halli_RawData_12_zones/ImpoudmentSizeIndex/dam_height_0_5m.tif with resolution 0.5x0.5.


### Finding locations with python

In [24]:
from osgeo import gdal
print(gdal.__file__)

h:\PurOja\.venv\Lib\site-packages\osgeo\gdal.py


In [25]:
import os
from osgeo import gdal

print("GDAL version:", gdal.__version__)
print("GDAL installation path:", gdal.__file__)

GDAL version: 3.8.4
GDAL installation path: h:\PurOja\.venv\Lib\site-packages\osgeo\gdal.py


### Merging files as vrt
Example, if you clipped the images to smaller ones while processing, this will combine them as virtual image. 

In [12]:
import os
from osgeo import gdal

def generate_file_list(path, pattern, output_file):
    files = [os.path.join(path, file) for file in os.listdir(path) if pattern in file]
    with open(output_file, 'w') as f:
        for file in files:
            f.write(f"{file}\n")

def create_vrt_for_files(file_list, vrt_output):
    # Open file list
    with open(file_list, 'r') as f:
        file_paths = [line.strip() for line in f]
    
    # Create VRT
    vrt_options = gdal.BuildVRTOptions(resampleAlg='bilinear')
    gdal.BuildVRT(vrt_output, file_paths, options=vrt_options)

def main():
    base_path = "E:/Gradu/ImpoudmentSizeIndex/splitted_vector2/output/"
    output_dir = "E:/Gradu/ImpoudmentSizeIndex/splitted_vector2/merged/"
    patterns = {
        "area": "area_filled_dem_resampled_Halli_",
        "max": "max_filled_dem_resampled_Halli_",
        "mean": "mean_filled_dem_resampled_Halli_",
        "dam_height": "dam_height_filled_dem_resampled_Halli_",
        "volume": "volume_filled_dem_resampled_Halli_"
    }

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for key, pattern in patterns.items():
        file_list = f"{key}_file_list.txt"
        vrt_output = os.path.join(output_dir, f"{key}_merged_output.vrt")
        
        generate_file_list(base_path, pattern, file_list)
        create_vrt_for_files(file_list, vrt_output)
        print(f"Created VRT for {key}: {vrt_output}")

if __name__ == "__main__":
    main()

Created VRT for area: E:/Gradu/ImpoudmentSizeIndex/splitted_vector2/merged/area_merged_output.vrt
Created VRT for max: E:/Gradu/ImpoudmentSizeIndex/splitted_vector2/merged/max_merged_output.vrt
Created VRT for mean: E:/Gradu/ImpoudmentSizeIndex/splitted_vector2/merged/mean_merged_output.vrt
Created VRT for dam_height: E:/Gradu/ImpoudmentSizeIndex/splitted_vector2/merged/dam_height_merged_output.vrt
Created VRT for volume: E:/Gradu/ImpoudmentSizeIndex/splitted_vector2/merged/volume_merged_output.vrt


### Creating a file if one is missing
Example, many times it's about misspelling something, chcek the names and the paths. Copy past is useful when to get the names and the locations right every time.

In [11]:
from osgeo import gdal

def create_vrt_for_dam_height(base_path, output_vrt):
    # List of TIFF files for 'dam_height' pattern
    pattern = "dam_height_merged_output_"
    file_list = [os.path.join(base_path, file) for file in os.listdir(base_path) if pattern in file]

    if not file_list:
        print("No files found with the specified pattern.")
        return

    # Create VRT
    vrt_options = gdal.BuildVRTOptions(resampleAlg='bilinear')
    gdal.BuildVRT(output_vrt, file_list, options=vrt_options)
    print(f"Created VRT for dam_height: {output_vrt}")

# Specify the base path and output VRT file path
base_path = "E:/Gradu/ImpoudmentSizeIndex/splitted_vector2/output/"
output_vrt = "E:/Gradu/ImpoudmentSizeIndex/splitted_vector2/merged/dam_height_merged_output.vrt"

create_vrt_for_dam_height(base_path, output_vrt)

No files found with the specified pattern.


### Making 1m resolution

In [18]:
import subprocess

# Define input and output file paths
input_file = "E:/Gradu/ImpoudmentSizeIndex/DEM_0_5m_nn.tif"
output_file = "E:/Gradu/ImpoudmentSizeIndex/DEM_1m_nn.tif"

# Define the gdalwarp command and its parameters
gdalwarp_command = [
    "gdalwarp",          # The command
    "-tr", "1.0", "1.0",  # Target resolution
    "-r", "bilinear",     # Resampling method
    input_file,          # Input raster file
    output_file          # Output raster file
]

try:
    # Run the gdalwarp command
    subprocess.run(gdalwarp_command, check=True)
    print("gdalwarp executed successfully.")
except subprocess.CalledProcessError as e:
    print(f"An error occurred: {e}")
except FileNotFoundError:
    print("gdalwarp not found. Make sure GDAL is installed and in your PATH.")

gdalwarp executed successfully.
